# Connection à la base de données 'munic'

In [10]:
import pandas as pd
import pymongo

client = pymongo.MongoClient('localhost')
collection = client.fcd.assets

print('done')

done


# Lecture dans la base

In [11]:
cursor = collection.find()
entries = list(cursor)

#entries[:]

df = pd.DataFrame(entries)
df.head()

,_id,asset_id,recorded_at,location
0,5f16a5bd9c984c3e2ad2fc23,14429000032073,1.583104e+09,"{'geo': {'coordinates': [-114.028, 50.9551], '..."
1,5f16a5bd9c984c3e2ad2fc24,14429000032073,1.583104e+09,"{'geo': {'coordinates': [-114.028, 50.9554], '..."
2,5f16a5bd9c984c3e2ad2fc25,14429000032073,1.583104e+09,"{'geo': {'coordinates': [-114.028, 50.9558], '..."
3,5f16a5bd9c984c3e2ad2fc26,14429000032073,1.583104e+09,"{'geo': {'coordinates': [-114.026, 50.9554], '..."
4,5f16a5bd9c984c3e2ad2fc27,14429000032073,1.583104e+09,"{'geo': {'coordinates': [-114.026, 50.9544], '..."


# Entrée 1 : géo

In [38]:
lon = float(input("longitude -> "))
lat = float(input("latitude -> "))
rad = float(input("rayon (km) -> "))

longitude -> -114.028
latitude -> 50.955
rayon (km) -> 5


# Entrée 2 : date

In [41]:
from datetime import timedelta
from datetime import datetime

year = int(input("Annee -> "))
month = int(input("Mois -> "))
day = int(input("Jour -> "))
hour = int(input("Heure -> "))
minutes = int(input("Minutes -> "))
seconds = int(input("Secondes -> "))
date = datetime(year, month, day, hour, minutes, seconds)

Annee -> 2020
Mois -> 3
Jour -> 5
Heure -> 14
Minutes -> 16
Secondes -> 2


# Entrée 3 : période

In [46]:
T = int(input("Période (minutes) -> "))
delta = timedelta(minutes = T)

datemin = date - delta
datemax = date + delta

datemin = datetime.timestamp(datemin)
datemax = datetime.timestamp(datemax)

Période (minutes) -> 3


In [62]:
import pprint

def km_to_radian(km):
    miles = km / 1.60934 # in miles
    return miles / 3963.2


# QUERY 1
# Calculer la densité [nombre de points] sur une Zone géographique Z et une période T 

# points dans une zone Z
query01 = {
    "location.geo": {
        "$nearSphere": {
            "$geometry": {
                "type": "Point",
                "coordinates": [ lon, lat]
            },
            "$minDistance": 0,
            "$maxDistance": rad*1000 # mètres
        }
    }
}

# points à une période T
query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}


# requête finale
query = {'$and': [query01, query02]}


result = collection.find(query)

for doc in result:
    print()
    pprint.pprint(doc)
    
print('\ndone')


{'_id': ObjectId('5f16a5bf9c984c3e2ad4051e'),
 'asset_id': '14429000032073',
 'location': {'geo': {'coordinates': [-114.066, 50.9304], 'type': 'Point'}},
 'recorded_at': 1583414329.0}

{'_id': ObjectId('5f16a5bf9c984c3e2ad40526'),
 'asset_id': '14429000032073',
 'location': {'geo': {'coordinates': [-114.067, 50.9308], 'type': 'Point'}},
 'recorded_at': 1583414336.0}

{'_id': ObjectId('5f16a5bf9c984c3e2ad40529'),
 'asset_id': '14429000032073',
 'location': {'geo': {'coordinates': [-114.067, 50.9308], 'type': 'Point'}},
 'recorded_at': 1583414337.0}

{'_id': ObjectId('5f16a5bf9c984c3e2ad4047d'),
 'asset_id': '14429000014790',
 'location': {'geo': {'coordinates': [-113.972, 50.9503], 'type': 'Point'}},
 'recorded_at': 1583414030.0}

{'_id': ObjectId('5f16a5bf9c984c3e2ad40479'),
 'asset_id': '14429000026877',
 'location': {'geo': {'coordinates': [-113.972, 50.9502], 'type': 'Point'}},
 'recorded_at': 1583414023.0}

{'_id': ObjectId('5f16a5bf9c984c3e2ad40475'),
 'asset_id': '14429000026877